# AOG Real-time Speech Recognition Example

This notebook demonstrates how to use AOG Speech-to-Text-WS API for real-time speech recognition.

⚠️ **Important**: This service uses WebSocket protocol, audio must be in PCM format

## Table of Contents
1. [Environment Setup](#1-Environment Setup)
2. [Helper Functions](#2-Helper Functions)
3. [Basic Real-time Recognition](#3-Basic Real-time Recognition)
4. [Handle Different Audio Formats](#4-Handle Different Audio Formats)
5. [Batch Processing](#5-Batch Processing)

## 1. Environment Setup

In [ ]:
import websocket
import json
import threading
import time
from pathlib import Path
from typing import Optional, List, Dict
from pydub import AudioSegment
import io

# AOG service configuration
AOG_WS_URL = "ws://localhost:16688/aog/v0.2/services/speech-to-text-ws"
DEFAULT_MODEL = "NamoLi/whisper-large-v3-ov"

print("✅ Configuration complete")
print(f"📍 WebSocket 地址: {AOG_WS_URL}")
print(f"🤖 Default model: {DEFAULT_MODEL}")
print("⚠️  Note: 音频必须为 PCM 格式(16000Hz, Mono, 16-bit)")

## 2. Helper Functions

In [ ]:
class RealtimeSTT:
    """实时语音识别客户端"""
    
    def __init__(self, ws_url: str, model: str = DEFAULT_MODEL):
        self.ws_url = ws_url
        self.model = model
        self.ws = None
        self.task_id = None
        self.results = []
        self.is_running = False
        
    def on_message(self, ws, message):
        """处理服务端消息"""
        try:
            data = json.loads(message)
            header = data.get("header", {})
            event = header.get("event")
            
            if event == "task-started":
                self.task_id = header.get("task_id")
                print(f"✅ Task started successfully: {self.task_id}")
                
            elif event == "result-generated":
                payload = data.get("payload", {})
                output = payload.get("output", {})
                sentence = output.get("sentence", {})
                text = sentence.get("text", "")
                begin_time = sentence.get("begin_time")
                end_time = sentence.get("end_time")
                
                if text:
                    result = {
                        "text": text,
                        "begin_time": begin_time,
                        "end_time": end_time
                    }
                    self.results.append(result)
                    print(f"📝 [{begin_time}ms]: {text}")
                    
            elif event == "task-finished":
                print("✅ Task completed")
                self.is_running = False
                
            elif event == "task-failed":
                error_msg = header.get("error_message", "Unknown error")
                print(f"❌ Task failed: {error_msg}")
                self.is_running = False
                
        except Exception as e:
            print(f"❌ 处理消息失败: {e}")
    
    def on_error(self, ws, error):
        """处理错误"""
        print(f"❌ WebSocket 错误: {error}")
        self.is_running = False
    
    def on_close(self, ws, close_status_code, close_msg):
        """处理连接关闭"""
        print(f"🔌 WebSocket 连接已关闭")
        self.is_running = False
    
    def on_open(self, ws):
        """连接建立后发送 run-task 指令"""
        print("🔌 WebSocket 连接已建立")
        
        run_task_msg = {
            "task": "speech-to-text-ws",
            "action": "run-task",
            "model": self.model,
            "parameters": {
                "format": "pcm",
                "sample_rate": 16000,
                "language": "zh",
                "use_vad": True,
                "return_format": "text"
            }
        }
        
        ws.send(json.dumps(run_task_msg))
        print("📤 已发送 run-task 指令")
        self.is_running = True
    
    def connect(self):
        """建立 WebSocket 连接"""
        self.ws = websocket.WebSocketApp(
            self.ws_url,
            on_open=self.on_open,
            on_message=self.on_message,
            on_error=self.on_error,
            on_close=self.on_close
        )
        
        # 在后台线程中运行
        wst = threading.Thread(target=self.ws.run_forever)
        wst.daemon = True
        wst.start()
        
        # 等待连接建立
        time.sleep(2)
    
    def send_audio(self, audio_data: bytes, chunk_size: int = 1024):
        """发送音频数据"""
        if not self.ws or not self.is_running:
            print("❌ WebSocket 未连接或任务未运行")
            return
        
        print(f"📤 开始发送音频数据 ({len(audio_data)} 字节)...")
        
        # 分块发送
        for i in range(0, len(audio_data), chunk_size):
            chunk = audio_data[i:i+chunk_size]
            self.ws.send(chunk, opcode=websocket.ABNF.OPCODE_BINARY)
            time.sleep(0.01)  # 模拟实时流
        
        print("✅ 音频数据发送完成")
    
    def finish(self):
        """结束任务"""
        if not self.ws or not self.task_id:
            print("❌ 无法结束任务:任务未启动")
            return
        
        finish_msg = {
            "task": "speech-to-text-ws",
            "action": "finish-task",
            "task_id": self.task_id,
            "model": self.model
        }
        
        self.ws.send(json.dumps(finish_msg))
        print("📤 已发送 finish-task 指令")
        
        # 等待Task completed
        time.sleep(2)
    
    def close(self):
        """关闭连接"""
        if self.ws:
            self.ws.close()
        print("🔌 连接已关闭")
    
    def get_results(self) -> List[Dict]:
        """获取Recognition result"""
        return self.results

def convert_to_pcm(audio_path: str) -> Optional[bytes]:
    """将音频文件转换为 PCM 格式"""
    try:
        print(f"🔄 转换音频为 PCM 格式: {audio_path}")
        
        # 加载音频
        audio = AudioSegment.from_file(audio_path)
        
        # 转换为 PCM: 16000Hz, Mono, 16-bit
        audio = audio.set_frame_rate(16000)
        audio = audio.set_channels(1)
        audio = audio.set_sample_width(2)  # 16-bit = 2 bytes
        
        # 导出为 PCM
        pcm_data = audio.raw_data
        
        print(f"✅ 转换完成: {len(pcm_data)} 字节")
        return pcm_data
        
    except Exception as e:
        print(f"❌ 转换失败: {e}")
        return None

print("✅ Helper Functions定义完成")

## 3. Basic Real-time Recognition

In [ ]:
# 准备音频文件
audio_path = "path/to/your/audio.wav"

if not Path(audio_path).exists():
    print(f"⚠️  请先准备一个音频文件,并将路径设置为: {audio_path}")
    print("💡 Tip: 支持 WAV、MP3 等格式,会自动转换为 PCM")
else:
    print("✅ 找到音频文件")
    print(f"📁 路径: {audio_path}")

In [ ]:
# 转换音频为 PCM
pcm_data = convert_to_pcm(audio_path)

if pcm_data:
    # 创建客户端
    client = RealtimeSTT(AOG_WS_URL)
    
    # 连接
    client.connect()
    
    # 发送音频
    client.send_audio(pcm_data)
    
    # 等待处理
    time.sleep(3)
    
    # 结束任务
    client.finish()
    
    # 获取结果
    results = client.get_results()
    
    print(f"\n📊 Recognition result汇总:")
    print("=" * 50)
    for i, result in enumerate(results, 1):
        print(f"{i}. [{result['begin_time']}ms - {result['end_time']}ms]: {result['text']}")
    
    # 关闭连接
    client.close()

## 4. Handle Different Audio Formats

In [ ]:
# MP3 文件
mp3_path = "path/to/your/audio.mp3"

if Path(mp3_path).exists():
    pcm_data = convert_to_pcm(mp3_path)
    
    if pcm_data:
        client = RealtimeSTT(AOG_WS_URL)
        client.connect()
        client.send_audio(pcm_data)
        time.sleep(3)
        client.finish()
        
        results = client.get_results()
        print(f"\n📊 识别到 {len(results)} 段文本")
        
        client.close()

## 5. Batch Processing

In [ ]:
# Batch Processing多个音频文件
audio_files = [
    "path/to/audio1.wav",
    "path/to/audio2.wav",
    "path/to/audio3.wav"
]

all_results = []

for i, audio_file in enumerate(audio_files, 1):
    if not Path(audio_file).exists():
        print(f"⚠️  文件不存在: {audio_file}")
        continue
    
    print(f"\n🎤 处理文件 {i}/{len(audio_files)}: {audio_file}")
    print("=" * 50)
    
    pcm_data = convert_to_pcm(audio_file)
    
    if pcm_data:
        client = RealtimeSTT(AOG_WS_URL)
        client.connect()
        client.send_audio(pcm_data)
        time.sleep(3)
        client.finish()
        
        results = client.get_results()
        all_results.append({
            "file": audio_file,
            "results": results
        })
        
        client.close()
        
        # 避免请求过快
        if i < len(audio_files):
            time.sleep(2)

print(f"\n✅ 成功处理 {len(all_results)}/{len(audio_files)} 个文件")

## 🎉 Summary

Through this notebook, you have learned:

1. ✅ 使用 WebSocket 连接实时语音识别服务
2. ✅ 将音频转换为 PCM 格式
3. ✅ 流式发送音频数据
4. ✅ 处理实时Recognition result
5. ✅ Batch Processing多个音频文件

### 关键要点

**音频格式要求**:
- 格式: PCM
- 采样率: 16000 Hz
- 声道: 单声道(Mono)
- 位深度: 16-bit

**WebSocket 通信流程**:
1. 连接 WebSocket
2. 发送 run-task 指令
3. 接收 task-started 事件
4. 发送 PCM 音频数据
5. 接收 result-generated 事件
6. 发送 finish-task 指令
7. 接收 task-finished 事件

**事件类型**:
- `task-started`: 任务启动
- `result-generated`: Recognition result
- `task-finished`: Task completed
- `task-failed`: Task failed

### Next Steps

- 尝试不同语言音频
- 调整 VAD Args
- Check out [Speech-to-Text](../speech-to-text/) 服务(批量识别)
- Read [AOG API 文档](../../docs/)